In [1]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet



# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

In [3]:
war_df = mss_df.loc[mss_df['UniqueID'].str.contains("WAR")].copy()

In [ ]:
war_df['Reporting Period'] = war_df['Reporting Period'].fillna(0)
war_df.loc[war_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = war_df.loc[pd.to_numeric(war_df['Reporting Period'], errors='coerce').isnull() & war_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
war_df['trial'] = war_df.index.map(s)
war_df.loc[pd.to_numeric(war_df['Reporting Period'], errors='coerce').isnull() & war_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = war_df.loc[pd.to_numeric(war_df['Reporting Period'], errors='coerce').isnull() & war_df['Reporting Period'].str.contains("RP"), 'trial']
war_df['Reporting Period'] = pd.to_numeric(war_df['Reporting Period'], downcast='integer')

In [9]:
war_df['Current RP Expected GT'] = pd.to_numeric(war_df['Current RP Expected GT'], errors='coerce')

In [13]:
expected_in_prog = war_df[['UniqueID', 'Current RP Expected GT', 'Reporting Period', 'Status', 'Future GT']].copy()

In [17]:
expected_in_prog = expected_in_prog[expected_in_prog['Current RP Expected GT'].notna()]
expected_in_prog

,UniqueID,Current RP Expected GT,Reporting Period,Status,Future GT
0,WAR241025TT01,4528.00,9,PENDING,0.0
1,WAR241025HD17,4689.54,9,PENDING,0.0
2,WAR241025HD16,5174.26,9,PENDING,0.0
3,WAR241025HD15,350.58,9,PENDING,0.0
4,WAR241025HD14,4417.97,9,PENDING,0.0
...,...,...,...,...,...
513,WAR220801CC03,10324.00,8,COMPLETED,
515,WAR220729TH01,17604.00,8,WITHDRAWN,
516,WAR220728TT01,3088.50,8,COMPLETED,
517,WAR220728TH02,3088.50,8,COMPLETED,


In [19]:
expected_minus_9 = expected_in_prog[expected_in_prog['Reporting Period'] != 9]
expected_minus_9

,UniqueID,Current RP Expected GT,Reporting Period,Status,Future GT
181,WAR230330TD02,12500.0,8,COMPLETED,
182,WAR230330TD01,2770.0,8,COMPLETED,
343,WAR240618BI03,11630.0,8,IN PROCESS,0.0
344,WAR240618BI02,1724.0,8,OPEN,0.0
345,WAR240618BI01,682.0,8,OPEN,0.0
...,...,...,...,...,...
513,WAR220801CC03,10324.0,8,COMPLETED,
515,WAR220729TH01,17604.0,8,WITHDRAWN,
516,WAR220728TT01,3088.5,8,COMPLETED,
517,WAR220728TH02,3088.5,8,COMPLETED,


Ok, now I need to translate this to TicketIDs

In [20]:
ticket_df = pd.read_excel("ticketid_241030.xlsx")

In [21]:
ticket_df

,Unnamed: 0,TicketID,UniqueID
0,0,WAR230113TD03,WAR230113TD03
1,1,WAR230113TD04,WAR230113TD04
2,2,WAR230203TTXX,WAR230203TT01
3,3,WAR230203TTXX,WAR230203TT02
4,4,WAR230303TH01,WAR230303TH01
...,...,...,...
291,291,WAR240529BI07,WAR240529BI07
292,292,WAR240529TT01,WAR240529TT01
293,293,WAR240719CC01,WAR240719CC01
294,294,WAR240719CC02,WAR240719CC02


In [22]:
merged_non9_expected = pd.merge(expected_minus_9, ticket_df, on='UniqueID', how='left')
merged_non9_expected

,UniqueID,Current RP Expected GT,Reporting Period,Status,Future GT,Unnamed: 0,TicketID
0,WAR230330TD02,12500.0,8,COMPLETED,,236,WAR230330TDXX
1,WAR230330TD01,2770.0,8,COMPLETED,,235,WAR230330TDXX
2,WAR240618BI03,11630.0,8,IN PROCESS,0.0,96,WAR240618BI03
3,WAR240618BI02,1724.0,8,OPEN,0.0,95,WAR240618BI02
4,WAR240618BI01,682.0,8,OPEN,0.0,94,WAR240618BI01
...,...,...,...,...,...,...,...
161,WAR220801CC03,10324.0,8,COMPLETED,,210,WAR220801CC03
162,WAR220729TH01,17604.0,8,WITHDRAWN,,207,WAR220729TH01
163,WAR220728TT01,3088.5,8,COMPLETED,,206,WAR220728XXXX
164,WAR220728TH02,3088.5,8,COMPLETED,,205,WAR220728XXXX


In [25]:
trial_expected_non9 = merged_non9_expected.groupby(['TicketID', 'Reporting Period'])['Current RP Expected GT'].sum().reset_index()

In [26]:
trial_expected_non9

,TicketID,Reporting Period,Current RP Expected GT
0,WAR211101CC01,6,0.0
1,WAR220728XXXX,8,6177.0
2,WAR220729TH01,8,17604.0
3,WAR220801CC01,8,3268.0
4,WAR220801CC03,8,10324.0
...,...,...,...
96,WAR240606TH03,8,0.0
97,WAR240606TH04,8,0.0
98,WAR240618BI01,8,682.0
99,WAR240618BI02,8,1724.0


In [29]:
# Somethign to think about for DLT is that the volume might be registered for next RP
merged_non9_expected.loc[merged_non9_expected['Current RP Expected GT'] ==  0, 'Reporting Period'].unique().tolist()

[8, 6]

In [30]:
trial_expected_non9

,TicketID,Reporting Period,Current RP Expected GT
0,WAR211101CC01,6,0.0
1,WAR220728XXXX,8,6177.0
2,WAR220729TH01,8,17604.0
3,WAR220801CC01,8,3268.0
4,WAR220801CC03,8,10324.0
...,...,...,...
96,WAR240606TH03,8,0.0
97,WAR240606TH04,8,0.0
98,WAR240618BI01,8,682.0
99,WAR240618BI02,8,1724.0


Now I need to get the RP9 expected from the worked up sheet I made

In [31]:
rp9_df = pd.read_excel("rp9_investigate_matchesRob.xlsx", sheet_name='Sheet2')

In [35]:
rp9_df.columns

Index(['TicketID', 'RP9 Expected', 'Reporting Period'], dtype='object')

In [37]:
rp9_df['Reporting Period'] = 9
rp9_df['Current RP Expected GT'] = rp9_df['RP9 Expected']
rp9_df

,TicketID,RP9 Expected,Reporting Period,Current RP Expected GT
0,WAR230908XXXX,9019.000000,9,9019.000000
1,WAR231107XXXX,12289.710000,9,12289.710000
2,WAR240521CCXX,9814.680000,9,9814.680000
3,WAR240521TN01,2918.650000,9,2918.650000
4,WAR240521TT01,6756.530000,9,6756.530000
5,WAR240627BI02,23703.790000,9,23703.790000
6,WAR240702BI01,16460.500000,9,16460.500000
7,WAR240702BI02,16748.400000,9,16748.400000
8,WAR240722BIXX,15434.600000,9,15434.600000
9,WAR240729TH01,7168.260000,9,7168.260000
